In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [25]:
data.info()
# null 이 아닌값들이 몇개 있는지 나온다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [26]:
print(data.isnull().sum())
# 각 열마다 null 값이 몇개씩 있는지 나온다.

# 결측값이 상당히 많다.
# 홈페이지 이런거는 별로 관련 없다 생각하면 과감하게 빼버려도 된다.
# id 같은 것도 마찬가지
# original language 같은 경우 결측값이 11개
# 이 11개의 결측값을 어떻게 채워넣을 것인가 고민해야 함
# overview 줄거리가 954개의 결측값이 존재함. 어떻게 처리, 대체할 것이냐
# overview 는 결측치 대체가 불가능하다. 그래서 overview가 없는 것들은
# 그냥 빼야 한다.

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64


In [27]:
data[ data['overview'].isnull() ]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
32,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...",NaN,78802,tt0114952,fr,"Guillaumet, les ailes du courage",NaN,...,1996-09-18,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wings of Courage,False,6.8,4.0
300,False,NaN,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,161495,tt0114296,sv,Roommates,NaN,...,1995-03-01,12400000.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Roommates,False,6.4,7.0
634,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,287305,tt0117312,de,Peanuts – Die Bank zahlt alles,NaN,...,1996-03-21,0.0,NaN,[],Released,NaN,Peanuts – Die Bank zahlt alles,False,4.0,1.0
635,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,339428,tt0116485,de,Happy Weekend,NaN,...,1996-03-14,65335.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Happy Weekend,False,0.0,0.0
641,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,10801,tt0117788,de,Das Superweib,NaN,...,1996-03-06,0.0,86.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,The Superwife,False,5.3,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45342,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,199887,tt1771636,en,Over/Under,NaN,...,2013-01-04,0.0,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Over/Under,False,4.0,2.0
45377,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}]",NaN,317389,tt0070695,es,Simbad e il califfo di Bagdad,NaN,...,1973-07-22,0.0,NaN,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Simbad e il califfo di Bagdad,False,0.0,0.0
45398,False,NaN,1254040,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://lmtr.fi/,468707,tt5742932,fi,Lauri Mäntyvaaran tuuheet ripset,NaN,...,2017-07-28,0.0,90.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,NaN,Thick Lashes of Lauri Mäntyvaara,False,8.0,1.0
45399,False,NaN,750000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,280422,tt3805180,ru,Все и сразу,NaN,...,2014-06-05,3.0,0.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,All at Once,False,6.0,4.0


In [28]:
data[ data['overview'].isnull() ].tail(3)
# 마지막 영화보면 줄거리가 NaN 이 있음
# 이런 것들을 어떻게 찾을까
# boolean 참조를 사용할 수 있음

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45398,False,NaN,1254040,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://lmtr.fi/,468707,tt5742932,fi,Lauri Mäntyvaaran tuuheet ripset,NaN,...,2017-07-28,0.0,90.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,NaN,Thick Lashes of Lauri Mäntyvaara,False,8.0,1.0
45399,False,NaN,750000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,280422,tt3805180,ru,Все и сразу,NaN,...,2014-06-05,3.0,0.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,All at Once,False,6.0,4.0
45410,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,449131,tt0321264,ru,Aprel,NaN,...,NaN,0.0,NaN,[],Released,NaN,Aprel,False,6.0,1.0


In [29]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 954


In [30]:
# 이런 null 인 것들을 모두 제거해야 한다.
# dropna 사용해 null 제거
data = data.dropna(subset = ['overview'])
data.tail(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [31]:
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 0


In [32]:
# 상위 2만개의 샘플만 data에 저장
data = data.head(20000)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 0 to 20135
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20000 non-null  object 
 1   belongs_to_collection  2407 non-null   object 
 2   budget                 20000 non-null  object 
 3   genres                 20000 non-null  object 
 4   homepage               3084 non-null   object 
 5   id                     20000 non-null  object 
 6   imdb_id                19994 non-null  object 
 7   original_language      19999 non-null  object 
 8   original_title         20000 non-null  object 
 9   overview               20000 non-null  object 
 10  popularity             19998 non-null  object 
 11  poster_path            19915 non-null  object 
 12  production_companies   19999 non-null  object 
 13  production_countries   19999 non-null  object 
 14  release_date           19984 non-null  object 
 15  revenue

In [35]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47661)


In [17]:
# 47661 차원에 영화벡터 20000 개.


# 차원이 너무 많다. 만약 한두개의 영화에만 나오는 단어고
# 나머지에는 한번도 안나오는 단어면
# 그것까지 전부 메모리에 올리는건 낭비일 수 있다.

# 일단 이 상태로 코사인 유사도 구한다면
# 20000 * 19999/2
# = 199990000 번 코사인 유사도를 구해야 함. 사이즈가 너무 크다.

# * 데이터가 1, 2, 3, 4 있을때
# 1을 기준으로 삼으면 2, 3, 4 를 연산해야 하니까
# n - 1 해서 19999 나옴
# 1, 2 계산이랑 2, 1 계산이랑 똑같으니까
# 2로 나눔

In [36]:
tfidf_matrix[0]

# 첫 번째 문서(=0번째 행) 의 줄거리를 뽑았는데
# 47661개의 차원이 있음

# 차원은 47661개가 있는데 0이 아닌값은 25개 있음
# 대부분은 값이 0이라는 뜻.
# 그래서 이걸 어떻게 사이즈를 줄일 것인가

# 단 한번밖에 언급되지 않은 데이터라면 코사인 유사도 계산에서 빼는게 좋다.


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 25 stored elements and shape (1, 47661)>

In [40]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

#  20,000행 20,000열의 행렬

코사인 유사도 연산 결과 : (20000, 20000)


In [41]:
cosine_sim[0]

# 첫번째는 1.
# 자기 자신과의 유사도니까 1이다
# 두번째와는 0.01577 나옴

array([1.        , 0.01577495, 0.        , ..., 0.        , 0.00826968,
       0.01721328])

In [44]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

# zip(data['title'], data.index)
# : 두 개의 시퀀스(title과 index)를 **쌍(pair)**으로 묶어줍니다.

# 예시:
# ["A", "B", "C"], [0, 1, 2] → ("A", 0), ("B", 1), ("C", 2)

# dict(...)
# zip으로 묶은 쌍(pair)을 **딕셔너리(dictionary)**로 만듭니다.

# 키(key): 'title'
# 값(value): 인덱스

4


In [45]:
dict(zip(data['title'], data.index))

{'Toy Story': 0,
 'Jumanji': 1,
 'Grumpier Old Men': 2,
 'Waiting to Exhale': 3,
 'Father of the Bride Part II': 4,
 'Heat': 14684,
 'Sabrina': 888,
 'Tom and Huck': 7,
 'Sudden Death': 8,
 'GoldenEye': 9,
 'The American President': 10,
 'Dracula: Dead and Loving It': 11,
 'Balto': 12,
 'Nixon': 13,
 'Cutthroat Island': 14,
 'Casino': 15,
 'Sense and Sensibility': 16,
 'Four Rooms': 17,
 'Ace Ventura: When Nature Calls': 18,
 'Money Train': 19,
 'Get Shorty': 20,
 'Copycat': 21,
 'Assassins': 22,
 'Powder': 23,
 'Leaving Las Vegas': 24,
 'Othello': 15347,
 'Now and Then': 26,
 'Persuasion': 14838,
 'The City of Lost Children': 28,
 'Shanghai Triad': 29,
 'Dangerous Minds': 30,
 'Twelve Monkeys': 31,
 'Babe': 33,
 'Carrington': 34,
 'Dead Man Walking': 35,
 'Across the Sea of Time': 36,
 'It Takes Two': 37,
 'Clueless': 38,
 'Cry, the Beloved Country': 39,
 'Richard III': 17719,
 'Dead Presidents': 41,
 'Restoration': 42,
 'Mortal Kombat': 43,
 'To Die For': 44,
 'How To Make An America

In [46]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

In [47]:
get_recommendations('The Dark Knight Rises')

,title
19286,The One Percent
4132,The Luzhin Defence
9844,Come and Get It
2817,The Fire Within
15225,Father and Son
4607,An American Rhapsody
9894,Winter Solstice
9033,Leningrad Cowboys Meet Moses
6651,Cobra
648,Billy's Holiday


In [48]:
get_recommendations('Toy Story')

,title
15348,Toy Story 3
2997,Toy Story 2
10301,The 40 Year Old Virgin
8327,The Champ
1071,Rebel Without a Cause
11399,For Your Consideration
1932,Condorman
3057,Man on the Moon
485,Malice
11606,Factory Girl
